<a href="https://colab.research.google.com/github/rerebebeccacca/20210624-deep-learning-class/blob/main/02_bidirectional_lstm_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bidirectional LSTM on IMDB

**Author:** [fchollet](https://twitter.com/fchollet)<br>

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, utils

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
INDEX_FROM = 3

## Build the model

In [ ]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(31))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()


In [ ]:
utils.plot_model(model, show_shapes=True)

## Load the IMDB movie review sentiment data

In [ ]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.imdb.load_data(
    num_words=max_features,
    start_char=1, # start tag index
    oov_char=2, # out-of-vocabulary index
    index_from=3, # index of actual word starts
)

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
len(x_train[5566]), x_train[5566]

In [ ]:
word2index2 = tf.keras.datasets.imdb.get_word_index()
index2word2 = {v: k for k, v in word2index2.items()}

index2word2[4]

In [ ]:
word2index = tf.keras.datasets.imdb.get_word_index() # word -> index
word2index = {k: v+INDEX_FROM for k,v in word2index.items()}
word2index["<PAD>"] = 0
word2index["<START>"] = 1
word2index["<UNK>"] = 2
word2index["<UNUSED>"] = 3
print(word2index)

In [ ]:
index2word = {v: k for k, v in word2index.items()} # index -> word
print(index2word)

In [ ]:
index2word[5]

In [ ]:
idx = np.random.randint(0, x_train.shape[0])
label = y_train[idx]
comment = x_train[idx]
print(f'idx: {idx}')
print('comment: ', comment)
label

In [ ]:
comment_words = [index2word[c] for c in comment]
' '.join(comment_words)

In [ ]:
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
x_train[5566]

## Train and evaluate the model

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_val, y_val))

### Exercise: Reuters classification

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.reuters.load_data()